In [257]:
from database import config
import mysql.connector
import numpy as np


db = mysql.connector.connect(**config)
cursor = db.cursor()

cursor.execute(
    """
    select gid from news_gpt.remote_question where questions is NULL and checked is NULL limit 10
    """
)
candidate =  cursor.fetchall()


In [ ]:
n_overlap = 0

while True:
        gid = candidate[np.random.randint(10)][0]
        # gid= '100000119'
        cursor.execute(
            f"""
            update news_gpt.remote_question set checked = '1' where gid = '{gid}' and checked is NULL
            """
        )
        if cursor.rowcount == 0:
            n_overlap +=1

            if n_overlap > 5:
                raise Exception("5번 이상 틀림")
            print('shit')
            pass
        else:
            print('fuck')
            db.commit()
            break

In [259]:
### 위에 둘을 합침 ####
from database import config
import mysql.connector
import numpy as np

def giveme_fresh_gid(multi):
    db = mysql.connector.connect(**config)
    cursor = db.cursor()

    cursor.execute(
        """
        select gid from news_gpt.remote_question where questions is NULL and checked is NULL limit 10
        """
    )
    candidate =  cursor.fetchall()
    if len(candidate)==0:
        print(f'{multi} end  print')
        return(f'end')
        # raise Exception("더 할거 없음")

    n_overlap = 0
    while True:
        gid = candidate[np.random.randint(len(candidate))][0]
        # gid= '100000119'
        cursor.execute(
            f"""
            update news_gpt.remote_question set checked = '1' where gid = '{gid}' and checked is NULL
            """
        )
        if cursor.rowcount == 0:
            n_overlap +=1

            if n_overlap > 10:
                print(f"{multi} 넘침")
                return giveme_fresh_gid(multi)
                # break
            print(f'{multi}  shit {n_overlap}')
            pass
        else:
            db.commit()
            return gid
            # break

# gid =giveme_fresh_gid()
# print(giveme_fresh_gid())
# print(gid)

In [179]:
#### 프롬프트 만들기 #####
import codecs


## document 
cursor.execute(
    f"""
    select title, createtime, content from news_gpt.remote_question where gid = "{gid}"
    """
)
title, createtime, content =  cursor.fetchall()[0]
content = codecs.decode(content, 'utf-8')

document = f"""title: {title}
create date: {createtime}
text:{content}
"""

## prompt sheet

prompt= """Instruction: Your task is two things.
1. show all full name of proper nouns(name and title of celebrity, organization name) from the document enclosed in three single quotation marks, which is used multiple time.
2. Present 15 questions for which you can hint from the document enclosed in three single quotation marks. Avoid questions whose answer exists in the form of a span in the text .
For the names of people and organizations, NEVER use abbreviations.
 use the following format for 2 tasks:
{"proper_noun":["Alpha", "Beta", "Gamma"],"questions":["qeustion1", "question2", "question15"]}
document : 
'''
"""+ document+"'''"

print(prompt)

Instruction: Your task is two things.
1. show all full name of proper nouns(name and title of celebrity, organization name) from the document enclosed in three single quotation marks, which is used multiple time.
2. Present 15 questions for which you can hint from the document enclosed in three single quotation marks. Avoid questions whose answer exists in the form of a span in the text .
For the names of people and organizations, NEVER use abbreviations.
 use the following format for 2 tasks:
{"proper_noun":["Alpha", "Beta", "Gamma"],"questions":["qeustion1", "question2", "question15"]}
document : 
'''
title: 이재웅 “‘타다 금지법’ 의결…혁신 금지 정부·국회는 죽었다”
create date: 2020-03-04 18:20:00
text:“혁신을 금지한 정부와 국회는 죽었다.” 4일 국회 법제사법위원회 전체회의에서 이른바 ‘타다 금지법(여객자동차운수사업법 개정안)’이 통과되자 이재웅 쏘카 대표는 자신의 페이스북에 “국회 법사위도 과거의 시간으로 되돌아가겠다고 결정을 했다. 새로운 꿈을 꿀 기회조차 앗아간 정부와 국회는 죽었다”고 적었다. 개정안은 관광 목적으로 11~15인승 렌터카를 빌릴 경우 6시간 이상 사용 또는 대여, 반납 장소가 공항이나 항만일 때만 사업자가 운전자를 알선할 수 있도록 했다. 이 대표는 국회 본회의에서 타다 금지법이 통과되면 서비스를 사실상 중단해야한다는 뜻

In [228]:
#### 프롬프트 만들기 #####
import codecs

def giveme_prompt(gid):
    ## document 
    db = mysql.connector.connect(**config)
    cursor = db.cursor()
    cursor.execute(
        f"""
        select title, createtime, content from news_gpt.remote_question where gid = "{gid}"
        """
    )
    title, createtime, content =  cursor.fetchall()[0]
    content = codecs.decode(content, 'utf-8')

    document = f"""title: {title}
create date: {createtime}
text:{content}
"""

    ## prompt sheet

    prompt= """Instruction: Your task is two things.
1. show all full name of proper nouns(name and title of celebrity, organization name) from the document enclosed in three single quotation marks, which is used multiple time.
2. Present 15 questions for which you can hint from the document enclosed in three single quotation marks. Avoid questions whose answer exists in the form of a span in the text .
For the names of people and organizations, NEVER use abbreviations.
use the following format for 2 tasks:
{"proper_noun":["Alpha", "Beta", "Gamma"],"questions":["qeustion1", "question2", "question15"]}
document : 
'''
"""+ document+"'''"
    return prompt


In [229]:
print(giveme_prompt(giveme_fresh_gid()))

Instruction: Your task is two things.
1. show all full name of proper nouns(name and title of celebrity, organization name) from the document enclosed in three single quotation marks, which is used multiple time.
2. Present 15 questions for which you can hint from the document enclosed in three single quotation marks. Avoid questions whose answer exists in the form of a span in the text .
For the names of people and organizations, NEVER use abbreviations.
use the following format for 2 tasks:
{"proper_noun":["Alpha", "Beta", "Gamma"],"questions":["qeustion1", "question2", "question15"]}
document : 
'''
title: 세계 절반이 한국에 ‘빗장’…한국발 입국제한 93곳
create date: 2020-03-04 22:16:00
text:﻿(서울=뉴스1) 민선희 기자 = 국내 신종 코로나바이러스 감염증(코로나19) 확산으로 인해 한국 방문자의 입국을 제한하는 국가·지역은 93곳까지 늘었다. 유엔 회원국(193개) 중 절반이 한국발 입국자에 대해 ‘빗장’을 걸어 잠갔다.외교부는 세계 각국과 다각도로 교섭을 진행하며 ‘코로나 외교 총력전’에 나섰다. 외교부 당국자는 “현재 1진1퇴의 치열한 공방전이 전개되고 있다”고 설명하기도 했다. 교섭 결과 우리국민 1000여명이 격리된 중국·베트남 일부 지역에서는 호텔·시설에서 격리하던 국민들을 자가격리로 전환한 성과도 있었다. 외교부 당국자는 항공편이 급격

In [260]:
import multiprocessing
import time
def each(result_list, i):
    print('yoyoyo')
    while True:
        
        gid = giveme_fresh_gid(i)
        print(f"{i}  {gid}")
        result_list.append(gid)
        if gid == 'end':
            return "done"

def do():
    print('here2')
    result_list = multiprocessing.Manager().list()
    processes = []
    for i in range(3):
        print(f'here   {i}')
        p = multiprocessing.Process(target=each, args=(result_list, i))
        processes.append(p)
        p.start()
    
    for p in processes:
        p.join()
    return list(result_list)
if __name__ == "__main__":
    print(do())
    

here2
here   0
here   1
here   2
[]
